Using Azure OpenAI Search

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.6.2"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.6.2-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.6.2-alpha"
#r "nuget: System.Linq.Async, 6.0.1"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
#pragma warning disable SKEXP0010
/* var config = new AzureOpenAIChatCompletionWithDataConfig
        {
            CompletionModelId = model,
            CompletionEndpoint = azureEndpoint,
            CompletionApiKey = apiKey,
            DataSourceEndpoint = "https://mropenaisearch.search.windows.net",
            DataSourceApiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_SEARCH_QUERYKEY"),
            DataSourceIndex = "properties"
        }; */
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            //.AddAzureOpenAIChatCompletion(config: config)
            .Build();

Installed Packages Microsoft.SemanticKernel, 1.6.2 Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.6.2-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.6.2-alpha System.Linq.Async, 6.0.1

In [14]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Connectors.AzureAISearch;
using Microsoft.SemanticKernel.Plugins.Memory;

#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0020
var memory = new MemoryBuilder()
    .WithAzureOpenAITextEmbeddingGeneration("mrtextembeddingada002", azureEndpoint, apiKey)
    .WithMemoryStore(new AzureAISearchMemoryStore("https://mropenaisearch.search.windows.net", Environment.GetEnvironmentVariable("AZURE_OPENAI_SEARCH_QUERYKEY")))
    .Build();
#pragma warning disable SKEXP0050  
kernel.Plugins.Clear();
kernel.ImportPluginFromObject(new TextMemoryPlugin(memory));         

In [8]:
var ask = "Looking for a small apartment with views of the ocean";
var function = kernel.CreateFunctionFromPrompt("Question: {{$input}}. Use data returned from Azure Search {{recall $input}} to answer. If Azure Search does not return any data, respond: I don't know.");
var response = await kernel.InvokeAsync(function, new() { ["input"] = ask });
Console.WriteLine($"Ask: {ask}");
Console.WriteLine($"Response: {response.GetValue<string>()}");
Console.WriteLine();

Ask: Looking for a small apartment with views of the ocean
Response: I'm sorry, but as an AI language model, I don't have access to real-time data or the ability to directly interact with Azure Search. Therefore, I won't be able to provide you with specific information about apartments with ocean views using Azure Search data. I suggest using a real estate website or contacting a local real estate agent who may be able to assist you in finding a small apartment with ocean views.



In [15]:
var questions = new[]
{
    "Looking for a small apartment with views of water"
};
foreach (var q in questions)
{
    var response = await memory.SearchAsync("properties", q, limit: 3, minRelevanceScore: 0.2, withEmbeddings: true).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}

Looking for a small apartment with views of water 
